# ☕️ RoastCo – Coffee Category Insights

As part of the Category Development team at RoastCo, my role was to analyse POS data to identify growth drivers in the coffee category. This project focuses on brand performance, promo effectiveness, and regional sales dynamics to support merchandising and pricing strategies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

sns.set(style='whitegrid')

In [ ]:
# Load dataset
df = pd.read_excel('roastco_data.xlsx')
df.head()

### 🧼 Data Cleaning
- Check for nulls and fix data types
- Convert promo_type and pack_size to category
- Extract year/month for time series

In [ ]:
df['promo_type'] = df['promo_type'].astype('category')
df['pack_size'] = df['pack_size'].astype('category')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.to_period('M')
df.isnull().sum()

### 📊 Units Sold by Brand

In [ ]:
brand_sales = df.groupby('brand')['units_sold'].sum().sort_values()
brand_sales.plot(kind='barh', figsize=(8,5), title='Total Units Sold by Brand')
plt.xlabel('Units Sold')
plt.show()

### 📊 Revenue by Promotion Type

In [ ]:
promo_rev = df.groupby('promo_type')['revenue'].mean().sort_values()
promo_rev.plot(kind='bar', figsize=(8,5), title='Average Revenue per Promo Type')
plt.ylabel('Avg Revenue')
plt.show()

### 🔍 Clustering Retailer Segments by Sales Mix

In [ ]:
pivot = pd.pivot_table(df, index='retailer', columns='brand', values='units_sold', aggfunc='sum', fill_value=0)
km = KMeans(n_clusters=3, random_state=42)
df_clusters = pivot.copy()
df_clusters['cluster'] = km.fit_predict(pivot)
df_clusters

### 📈 Regression: Price Impact on Units Sold

In [ ]:
enc = OneHotEncoder(drop='first')
X_cat = enc.fit_transform(df[['brand', 'promo_type', 'region']]).toarray()
X = np.concatenate([df[['price']].values, X_cat], axis=1)
y = df['units_sold']
reg = LinearRegression()
reg.fit(X, y)
print(f'R² Score: {reg.score(X, y):.2f}')

## ✅ Summary & Recommendations

- RoastCo Organic is outperforming on volume, especially in Western Canada.
- Endcap and BOGO promos offer the strongest ROI based on revenue per unit sold.
- Pricing sensitivity varies by brand—Original and Bold have stronger volume elasticity.
- Recommend scaling high-performing promos and optimising price tiers in top channels.